# Проверка работы FastText

In [1]:
import first_script

In [2]:
from tokenizer import FastTextTokenizer
from model import FastText
import os

## Основная особенность FastText - токенизатор, делящий слова на n-граммы

In [3]:
tokenizer = FastTextTokenizer(n_gram=3)

In [4]:
text_train = 'Привет, меня зовут Миша. Как тебя зовут? миша'
tokenizer.train(text_train)

In [5]:
text_test = 'Привет Миша, меня зовут'
print(tokenizer.encode(text_test))
print(tokenizer.encode(text_train))

[21, 33, 14, 10, 12, 31, 20, 26, 23, 29, 8, 18, 7, 37, 19, 9, 34, 36, 13, 27]
[21, 33, 14, 10, 12, 31, 8, 18, 7, 37, 19, 9, 34, 36, 13, 27, 20, 26, 23, 29, 16, 24, 35, 28, 22, 25, 32, 17, 9, 34, 36, 13, 27, 15, 11, 30, 23, 29]


In [6]:
text_test_idx = tokenizer.encode(text_test)
tokenizer.decode(text_test_idx)

['Привет', 'Миша', ',', 'меня', 'зовут']

In [7]:
tokenizer._tokenize(text_test)

['<Пр',
 'При',
 'рив',
 'иве',
 'вет',
 'ет>',
 '<Ми',
 'Миш',
 'иша',
 'ша>',
 '<,>',
 '<ме',
 'мен',
 'еня',
 'ня>',
 '<зо',
 'зов',
 'ову',
 'вут',
 'ут>']

In [8]:
text_test_idx = tokenizer.encode(text_test, add_special_tokens=True)
tokenizer.decode(text_test_idx, add_special_tokens=True)

['[BOP]', '[BOS]', 'Привет', 'Миша', ',', 'меня', 'зовут', '[EOS]', '[EOP]']

## Обучаем FastText

In [9]:
tokenizer = FastTextTokenizer(n_gram=3)

In [10]:
text = ""
with open('../data/voina_i_mir.txt', 'r') as f:
    text = f.read()
tokenizer.train(text)

In [11]:
text_idxs = tokenizer.encode(text, add_special_tokens=True)

### Параметры модели

In [12]:
embed_dim = 256
window_size = 20
device = 'cuda'

### Параметры обучения

In [13]:
num_epochs = 10
batch_size = 8000
num_workers = os.cpu_count()

### Загружаем модель и обучаем

In [14]:
model = FastText(embed_dim, window_size, len(tokenizer.words), device=device)
res = model.fit(text_idxs, batch_size, num_epochs, num_workers=num_workers)

Epoch [1/10, time: 1.049 minutes]:
                            Train Loss 12.470066, Train Perplexity 260423.805
                            Test Loss 11.771999, Test Perplexity 129572.890
Epoch [2/10, time: 2.121 minutes]:
                            Train Loss 10.908654, Train Perplexity 54647.216
                            Test Loss 10.299313, Test Perplexity 29712.211
Epoch [3/10, time: 3.284 minutes]:
                            Train Loss 9.655433, Train Perplexity 15606.340
                            Test Loss 9.351883, Test Perplexity 11520.498
Epoch [4/10, time: 4.516 minutes]:
                            Train Loss 8.930647, Train Perplexity 7560.156
                            Test Loss 8.800919, Test Perplexity 6640.342
Epoch [5/10, time: 5.810 minutes]:
                            Train Loss 8.452994, Train Perplexity 4689.092
                            Test Loss 8.403258, Test Perplexity 4461.579
Epoch [6/10, time: 7.098 minutes]:
                            Train Loss

### Визуализируем метрики

In [15]:
FastText.plot_metrics(*res)

### Слово разделяется на n-граммы

In [16]:
test_text = 'Добрый'
test_text_idx = tokenizer.encode(test_text)
print(test_text_idx)
dec_test_text_idx = tokenizer.decode(test_text_idx)
print(dec_test_text_idx)

[9350, 8445, 9140, 9411, 1992, 1483]
['Добрый']


In [17]:
embed = model[test_text_idx]

In [18]:
embed.shape

torch.Size([256])

### Благодаря разделению на n-граммы можно получить эмбеддинг любой фразы

In [19]:
test_text = 'Добрый и позитивный'
test_text_idx = tokenizer.encode(test_text)
print(test_text_idx)
dec_test_text_idx = tokenizer.decode(test_text_idx)
print(dec_test_text_idx)

[9350, 8445, 9140, 9411, 1992, 1483, 15133, 10123, 6346, 11099, 6757, 475, 13152, 10973, 8931, 8563, 1483]
['Добрый', 'и', 'позитивный']


In [20]:
embed = model[test_text_idx]

In [21]:
embed.shape

torch.Size([256])

### Почти любой...

In [22]:
test_text = 'кмлум'
test_text_idx = tokenizer.encode(test_text)
print(test_text_idx)
dec_test_text_idx = tokenizer.decode(test_text_idx)
print(dec_test_text_idx)

[0, 0, 0, 4127, 1805]
['м']


In [23]:
embed = model[test_text_idx]
embed.shape

torch.Size([256])

### Чтобы вычислить ближайшие слова к данному нужно немного магии (а именно добавить индексы каждого слова)

In [24]:
word1 = 'Андрей'
word1_idx = tokenizer.encode(word1)
all_words_idx = [tokenizer.encode(word) for word in tokenizer.real_words]
near1 = model.k_Nearest(all_words_idx, word1_idx, k=10, use_cosine=False)

In [25]:
print([tokenizer.real_words[near1_idx] for near1_idx in near1])

['Андрей', 'Андрее', 'Андрея', 'Андрею', 'Андреевской', 'Андреем', 'Андреич', 'Андреевич', 'Андреича', 'Андреевичу', 'Андреи']


In [26]:
near1, embeds = model.k_Nearest(all_words_idx, word1_idx, k=100, use_cosine=True, return_embed=True)
model.plot_embeddings([tokenizer.real_words[near1_idx] for near1_idx in near1], embeds)

In [29]:
[word for word in tokenizer.real_words if word[0] == '[' and word[-1] == ']']

['[24]',
 '[  вам одним могу признаться. Мои дети – обуза моего существования.]',
 '[  Женщина – подруга мужчины,]',
 '[  Вы говорите про Буонапарта?]',
 '[  Тому, кто храбрее всех показал себя во время войны,]',
 '[  словечек]',
 '[(сноска 42)]',
 '[  Были примеры – Шварценберг.]',
 '[(сноска 99)]',
 '[  Если бы ты имел веру, то обратился бы к Богу с молитвою, чтоб Он даровал тебе любовь, которую ты не чувствуешь, и молитва твоя была бы услышана.]',
 '[  из газа цвета металла,]',
 '[(сноска 58)]',
 '[  Раз он перейдет в гвардию…]',
 '[23]',
 '[(сноска 76)]',
 '[  почтеннейший]',
 '[(сноска 103)]',
 '[  Если вы так смотрите на предмет,]',
 '[  так проходит мирская слава.]',
 '[  Но, мой милый,]',
 '[(сноска 70)]',
 '[(сноска 33)]',
 '[  Александр, Россия, величие;]',
 '[  любимица,]',
 '[(сноска 35)]',
 '[(сноска 52)]',
 '[  убирайся,]',
 '[  Кодекса Наполеона и Юстиниана,]',
 '[81]',
 '[  Его сиятельство князь Курагин с сыном, сколько я слышала?]',
 '[  Никакого,]',
 '[  Но вы так чис

Повезло, случайно нашел фичу в токенизаторе (все в квадратных скобках он обходит).  
Почему повезло? фраза про сливки общества близка к слову Андрей - хорошая метафора!.. Оставлю тут)